In [24]:
import numpy as np

p = 10
sample_size = 100000
beta = 10000
sigma_q = 0.05
N = 20
#
lambda_ = 1/(2*sigma_q**2*beta)
sigma_sq = N/(2*beta)
#
import json
with open('data/gibbs_data_p_is_{0}.json'.format(p), 'r') as datafile:
    data = json.load(datafile)
i = 2
S = np.array(data[i]['S'])
X = S[:,:-1]
Y = S[:,-1]
################################################################
original_random_state = np.random.get_state()
np.random.seed(20230929)
#gaussian distribution sampler: normal(mean=0.0, variance_sqrt=1.0, size=None) s = np.random.normal(0,1)
#teacher model
#dimension of x: d（固定不变）10->5
d = 5
#number of training samples（固定不变）100->20
N = 20
#dimension of hypothesis space
p0 = 5 #dimension of teacher model（固定不变）
p = p #dimension of student model（可变）#p=200
p_max = 2000 #upper bound of dimension of student model（固定不变）
#point-wise activate function f:tanh
#variance of random noise added to y
sigma = 0.1

#random feature matrix
F0 = np.random.normal(0,1,(d,p0)) #生成teacher model的random feature matrix F0（固定不变）
F = np.random.normal(0,1,(d,p_max))[:,:p] #生成student model的random feature matrix F(d*p)。先按照p_max的大小生成，再根据当前维度p的不同截取前p列形成每个p对应的F。
    #注意，这个F对于一般的神经网络是可训练的，但是对于RFM为了简化直接设定为固定值。（固定不变）
#teacher parameter w(p) with lambda = ? until each dim of Y~1e0
lambda_ = 0.0001
#w_0 = np.random.normal(0,sigma/np.sqrt(lambda_*N),p)
w_0 = np.random.normal(0,1,p0) #生成teacher model的权重向量（固定不变）
#w_0 = np.concatenate((w_0,np.zeros(5)),axis = 0)
X_rf = np.tanh(X.dot(F)/np.sqrt(d))
np.random.set_state(original_random_state)
################################################################
B = X_rf
#
W = np.linalg.inv(lambda_*N*np.identity(p)+B.T.dot(B)).dot(B.T).dot(Y)
Sigma = sigma_sq*np.linalg.inv(lambda_*N*np.identity(p)+B.T.dot(B))

In [25]:
w_s = np.random.multivariate_normal(W,Sigma,10000)
print(w_s)
data[i]['w']

[[ 0.47021674  0.72240912 -0.07385836 ...  0.20311294 -0.02380433
  -0.05699841]
 [ 0.48293704  0.6810849  -0.10493172 ...  0.36532354  0.25950019
  -0.04956866]
 [ 0.42454941  0.78341897 -0.21180918 ...  0.22760612 -0.02753982
  -0.14836051]
 ...
 [ 0.44650187  0.70500228 -0.18957355 ...  0.23182196 -0.22995743
  -0.15954124]
 [ 0.45701894  0.59816182 -0.14388509 ...  0.4420852   0.08367528
  -0.14216228]
 [ 0.429471    0.63781701 -0.11628871 ...  0.21106492 -0.19318002
  -0.18130744]]


[0.49341270746819577,
 0.46468983344930076,
 0.007458138833922351,
 0.19441497125302565,
 0.20505205410057423,
 -0.41381544104515644,
 0.14688978710595987,
 0.4807835289179572,
 -0.08468309078461991,
 -0.04515984315972421]